# Aufgabenstellung
Alle Sätze in denen Ukraine und Russland vorkommen und Wandel in der Frequenz über Zeit visualisieren

## Plan

1. dataframe laden
2. tokenize in Sätze
    - Ergebnis: neue Tokens
    - Metadaten: Datum, Sprecher
3. Funktion: finde zwei Begriffe/lemmas innerhalb eines Tokens
4. Filter tokens
5. Count per period
    - auf der Basis der Metadaten
6. Visualise

# Data frame laden

In [3]:
import pandas as pd

In [4]:
df = pd.read_pickle('../data/reden-bundesregierung.p')

In [5]:
df.head(3)

,person,titel,datum,untertitel,url,anrede,rohtext,ort,doc_object,tokens,ntokens,lemmata,PER,ORG,LOC,MISC
0,Gerhard Schröder,Rede des Bundeskanzlers zur Eröffnung der 52. ...,2002-02-06,"Man kann diese Filmfestspiele nicht eröffnen, ...",http://archiv.bundesregierung.de/bpaexport/red...,"Sehr geehrte Frau Nair, liebe Mitglieder der J...","es ist angekündigt worden, man sollte im Beruf...",None,"(es, ist, angekündigt, worden, ,, man, sollte,...","[es, ist, angekündigt, worden, man, sollte, im...",1911,"[ankündigen, beruf, schauspieler, politiker, w...","[Hildegard Knef, Hildchen"", Billy Wilder, Audr...","[Berlin, Brückenschlags, Berliner Kinos, Jurym...","[Amerika, Deutschland, Berlin, Deutschland, La...","[Planet Kino, deutschen, deutschen, Irma La Do..."
1,Julian Nida-Rümelin,Redebeitrag von Staatsminister Nida-Rümelin in...,2002-02-01,"""Ich bin der Auffassung, wir müssen nicht nur ...",http://archiv.bundesregierung.de/bpaexport/red...,Meine sehr geehrten Damen und Herren!,"Frau Präsidentin! An Sie gerichtet, Herr Börn...",None,"(Frau, Präsidentin, !, , An, Sie, gerichtet, ...","[Frau, Präsidentin, , An, Sie, gerichtet, Her...",1319,"[frau, präsidentin, an, ich, richten, herr, bö...","[Otto, Filmkonzept, Otto, Höcherl, Otto]","[SPD-Fraktion, Bundestag, Goethe-Instituten, M...","[Deutschland, Pol, Bayern, Deutschland, Deutsc...","[Erinnerung., Filmpolitik, deutsche, Filmfesti..."
2,Gerhard Schröder,Interview mit Bundeskanzler Schröder in 'Berli...,2002-04-14,In dem Interview äußert sich Bundeskanzler Sch...,http://archiv.bundesregierung.de/bpaexport/red...,NaN,Frage (Peter Hahne): Bevor wir uns über den Au...,None,"(Frage, (, Peter, Hahne, ):, Bevor, wir, uns, ...","[Frage, Peter, Hahne, Bevor, wir, uns, über, d...",1512,"[frage, peter, hahn, bevor, aufbau, ost, arbei...","[Peter Hahne):, Saddam Hussein, Blair, Herrn F...","[EU-Ebene, Bundeswehr, Vereinten Nationen, Ver...","[Ferieninsel, Djerba, Deutschland, Israel, Deu...","[tunesischen, Deutsche, deutsche, tunesischen,..."


Neuer Index für den Dataframe damit wir später die Metadaten nachschlagen können. Neue Spalte und die soll Position.

In [7]:
df = df.assign(ID = range(2983)) # create column and add a counter
df.head(3)

,person,titel,datum,untertitel,url,anrede,rohtext,ort,doc_object,tokens,ntokens,lemmata,PER,ORG,LOC,MISC,ID
0,Gerhard Schröder,Rede des Bundeskanzlers zur Eröffnung der 52. ...,2002-02-06,"Man kann diese Filmfestspiele nicht eröffnen, ...",http://archiv.bundesregierung.de/bpaexport/red...,"Sehr geehrte Frau Nair, liebe Mitglieder der J...","es ist angekündigt worden, man sollte im Beruf...",None,"(es, ist, angekündigt, worden, ,, man, sollte,...","[es, ist, angekündigt, worden, man, sollte, im...",1911,"[ankündigen, beruf, schauspieler, politiker, w...","[Hildegard Knef, Hildchen"", Billy Wilder, Audr...","[Berlin, Brückenschlags, Berliner Kinos, Jurym...","[Amerika, Deutschland, Berlin, Deutschland, La...","[Planet Kino, deutschen, deutschen, Irma La Do...",0
1,Julian Nida-Rümelin,Redebeitrag von Staatsminister Nida-Rümelin in...,2002-02-01,"""Ich bin der Auffassung, wir müssen nicht nur ...",http://archiv.bundesregierung.de/bpaexport/red...,Meine sehr geehrten Damen und Herren!,"Frau Präsidentin! An Sie gerichtet, Herr Börn...",None,"(Frau, Präsidentin, !, , An, Sie, gerichtet, ...","[Frau, Präsidentin, , An, Sie, gerichtet, Her...",1319,"[frau, präsidentin, an, ich, richten, herr, bö...","[Otto, Filmkonzept, Otto, Höcherl, Otto]","[SPD-Fraktion, Bundestag, Goethe-Instituten, M...","[Deutschland, Pol, Bayern, Deutschland, Deutsc...","[Erinnerung., Filmpolitik, deutsche, Filmfesti...",1
2,Gerhard Schröder,Interview mit Bundeskanzler Schröder in 'Berli...,2002-04-14,In dem Interview äußert sich Bundeskanzler Sch...,http://archiv.bundesregierung.de/bpaexport/red...,NaN,Frage (Peter Hahne): Bevor wir uns über den Au...,None,"(Frage, (, Peter, Hahne, ):, Bevor, wir, uns, ...","[Frage, Peter, Hahne, Bevor, wir, uns, über, d...",1512,"[frage, peter, hahn, bevor, aufbau, ost, arbei...","[Peter Hahne):, Saddam Hussein, Blair, Herrn F...","[EU-Ebene, Bundeswehr, Vereinten Nationen, Ver...","[Ferieninsel, Djerba, Deutschland, Israel, Deu...","[tunesischen, Deutsche, deutsche, tunesischen,...",2


In [10]:
df_text = df[['ID', 'rohtext']]
df_text.head(3)

,ID,rohtext
0,0,"es ist angekündigt worden, man sollte im Beruf..."
1,1,"Frau Präsidentin! An Sie gerichtet, Herr Börn..."
2,2,Frage (Peter Hahne): Bevor wir uns über den Au...


# Neue Tokenisierung
Tokenisierung in Sätze oder arbiträre Windows. Wir müssen aber die Metadaten zu Datum behalten.

In [16]:
df_test = df[:3]
df_test.head(4)

,person,titel,datum,untertitel,url,anrede,rohtext,ort,doc_object,tokens,ntokens,lemmata,PER,ORG,LOC,MISC,ID
0,Gerhard Schröder,Rede des Bundeskanzlers zur Eröffnung der 52. ...,2002-02-06,"Man kann diese Filmfestspiele nicht eröffnen, ...",http://archiv.bundesregierung.de/bpaexport/red...,"Sehr geehrte Frau Nair, liebe Mitglieder der J...","es ist angekündigt worden, man sollte im Beruf...",None,"(es, ist, angekündigt, worden, ,, man, sollte,...","[es, ist, angekündigt, worden, man, sollte, im...",1911,"[ankündigen, beruf, schauspieler, politiker, w...","[Hildegard Knef, Hildchen"", Billy Wilder, Audr...","[Berlin, Brückenschlags, Berliner Kinos, Jurym...","[Amerika, Deutschland, Berlin, Deutschland, La...","[Planet Kino, deutschen, deutschen, Irma La Do...",0
1,Julian Nida-Rümelin,Redebeitrag von Staatsminister Nida-Rümelin in...,2002-02-01,"""Ich bin der Auffassung, wir müssen nicht nur ...",http://archiv.bundesregierung.de/bpaexport/red...,Meine sehr geehrten Damen und Herren!,"Frau Präsidentin! An Sie gerichtet, Herr Börn...",None,"(Frau, Präsidentin, !, , An, Sie, gerichtet, ...","[Frau, Präsidentin, , An, Sie, gerichtet, Her...",1319,"[frau, präsidentin, an, ich, richten, herr, bö...","[Otto, Filmkonzept, Otto, Höcherl, Otto]","[SPD-Fraktion, Bundestag, Goethe-Instituten, M...","[Deutschland, Pol, Bayern, Deutschland, Deutsc...","[Erinnerung., Filmpolitik, deutsche, Filmfesti...",1
2,Gerhard Schröder,Interview mit Bundeskanzler Schröder in 'Berli...,2002-04-14,In dem Interview äußert sich Bundeskanzler Sch...,http://archiv.bundesregierung.de/bpaexport/red...,NaN,Frage (Peter Hahne): Bevor wir uns über den Au...,None,"(Frage, (, Peter, Hahne, ):, Bevor, wir, uns, ...","[Frage, Peter, Hahne, Bevor, wir, uns, über, d...",1512,"[frage, peter, hahn, bevor, aufbau, ost, arbei...","[Peter Hahne):, Saddam Hussein, Blair, Herrn F...","[EU-Ebene, Bundeswehr, Vereinten Nationen, Ver...","[Ferieninsel, Djerba, Deutschland, Israel, Deu...","[tunesischen, Deutsche, deutsche, tunesischen,...",2


In [13]:
import spacy
nlp = spacy.load('de_core_news_md')  # Sprachmodell laden

In [14]:
#df_text['sents'] = df_text['rohtext'].apply(lambda x: list(nlp(x).sents)) # x is an arbitrary name


/var/folders/84/2v35331s5t97w8g_wvsp4nv80000gq/T/ipykernel_5670/1422796218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['sents'] = df_text['rohtext'].apply(lambda x: list(nlp(x).sents))


In [22]:
type(df_test.loc[0, 'doc_object'])

spacy.tokens.doc.Doc

In [52]:
# create a new dataframe
df_sents = pd.DataFrame('sent')
# create individual sentences and write them to the new dataframe
for doc in df_test.loc[:, 'doc_object']:
    # print(doc)
    for sent in doc.sents:
       df_sents.append(sent)

df_sents.head(3)

#print(df_test.loc[:, 'doc_object'].sents)

ValueError: DataFrame constructor not properly called!

# Jens Aurichs Lösung

Ich habe seine Lösung auf Listen als Input angepasst, damit sich mit Varianz von Schreibweisen umgehen lässt

In [69]:
def extract_sents(row, terms_1, terms_2):
    doc = row["doc_object"]
    # count number of sents
    nsents = len([sent for sent in doc.sents])
    row["nsents"] = nsents

    #find relevant sents
    relevant = [
        sent
        for sent in doc.sents
        # probably use lemma instead of text
        if any (i in sent.text for i in terms_1)
        and any (i in sent.text for i in terms_2)
    ]
    nsents_relevant = len(relevant)
    row["relevant_sents"] = relevant
    row["nsents_relevant"] = nsents_relevant
    return row

In [70]:
df_aurich = df.apply(lambda row: extract_sents(row, ["Ukraine"], ["Russland", "Rußland"]),
axis=1)

In [65]:
df_aurich.head(3)

,person,titel,datum,untertitel,url,anrede,rohtext,ort,doc_object,tokens,ntokens,lemmata,PER,ORG,LOC,MISC,ID,nsents,relevant_sents,nsents_relevant
0,Gerhard Schröder,Rede des Bundeskanzlers zur Eröffnung der 52. ...,2002-02-06,"Man kann diese Filmfestspiele nicht eröffnen, ...",http://archiv.bundesregierung.de/bpaexport/red...,"Sehr geehrte Frau Nair, liebe Mitglieder der J...","es ist angekündigt worden, man sollte im Beruf...",None,"(es, ist, angekündigt, worden, ,, man, sollte,...","[es, ist, angekündigt, worden, man, sollte, im...",1911,"[ankündigen, beruf, schauspieler, politiker, w...","[Hildegard Knef, Hildchen"", Billy Wilder, Audr...","[Berlin, Brückenschlags, Berliner Kinos, Jurym...","[Amerika, Deutschland, Berlin, Deutschland, La...","[Planet Kino, deutschen, deutschen, Irma La Do...",0,96,[],0
1,Julian Nida-Rümelin,Redebeitrag von Staatsminister Nida-Rümelin in...,2002-02-01,"""Ich bin der Auffassung, wir müssen nicht nur ...",http://archiv.bundesregierung.de/bpaexport/red...,Meine sehr geehrten Damen und Herren!,"Frau Präsidentin! An Sie gerichtet, Herr Börn...",None,"(Frau, Präsidentin, !, , An, Sie, gerichtet, ...","[Frau, Präsidentin, , An, Sie, gerichtet, Her...",1319,"[frau, präsidentin, an, ich, richten, herr, bö...","[Otto, Filmkonzept, Otto, Höcherl, Otto]","[SPD-Fraktion, Bundestag, Goethe-Instituten, M...","[Deutschland, Pol, Bayern, Deutschland, Deutsc...","[Erinnerung., Filmpolitik, deutsche, Filmfesti...",1,83,[],0
2,Gerhard Schröder,Interview mit Bundeskanzler Schröder in 'Berli...,2002-04-14,In dem Interview äußert sich Bundeskanzler Sch...,http://archiv.bundesregierung.de/bpaexport/red...,NaN,Frage (Peter Hahne): Bevor wir uns über den Au...,None,"(Frage, (, Peter, Hahne, ):, Bevor, wir, uns, ...","[Frage, Peter, Hahne, Bevor, wir, uns, über, d...",1512,"[frage, peter, hahn, bevor, aufbau, ost, arbei...","[Peter Hahne):, Saddam Hussein, Blair, Herrn F...","[EU-Ebene, Bundeswehr, Vereinten Nationen, Ver...","[Ferieninsel, Djerba, Deutschland, Israel, Deu...","[tunesischen, Deutsche, deutsche, tunesischen,...",2,77,[],0


In [67]:
nsents_relevant = df_aurich.nsents_relevant.sum() # pull a column from a dataframe and perform an operation on it
nsents_relevant

103